<a href="https://colab.research.google.com/github/ofekrosenfeld1/thesis/blob/main/vae_main_train_and_analyze_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#import relevant libraries
import torch
import torch.nn.functional as F
from torch import nn
import pandas as pd
import json
from torch.nn.utils.rnn import pad_sequence
import ast
from multiprocessing import Pool
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader
from __future__ import generator_stop

#mount drive

In [1]:
#connect to google drive in order to mount files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data PreProcessing for model


In [14]:
def load_data_and_selfies_indexing_jsons(file_name: str, idx_to_token_file: str, token_to_idx_file: str):
  """load the pre_processed data from chembl df, and also the pre-made jsons for selfies token/idx conversion"""
  #load the chembl pre-processed data into the notebook
  chembl_df = pd.read_csv('/content/drive/My Drive/chembl_relevant_info_with_numeric_selfies.csv')
  chembl_df = chembl_df.drop(columns=['Unnamed: 0','Unnamed: 0.1','ChEMBL ID'])
  #load the covnersion idxs jsons (selfies - numeric representation and vice versia)
  with open(f'/content/drive/My Drive/{idx_to_token_file}.json','r') as f:
    idx_to_tokens = json.load(f)
  with open(f'/content/drive/My Drive/{token_to_idx_file}.json','r') as f:
    tokens_to_idx = json.load(f)
  return chembl_df, idx_to_tokens, tokens_to_idx


In [29]:
def analyze_data_distr_for_optimal_paramaters(main_df : pd.DataFrame, percentile: float):
  """when training the VAE, we pad the sequences into constant value. Hence, we would not want a long sequence to let the generation go loose - hrut both training and generation. Its better to limit to the top x percentage of molecules with seq_len"""
  main_df['seq_len'] = main_df['numeric_selfies'].str.count(',') + 1
  seq_len = main_df['seq_len'].quantile(percentile)
  return seq_len

In [16]:
def fast_parse_and_pad(s, seq_len, pad_value = 0):
    """
    Converts stringified numeric SELFIE to a fixed-length tensor.
    Truncates if too long, pads with `pad_value` if too short.
    """
    tokens = list(map(int, s.strip('[]').replace(' ', '').split(',')))
    tokens = tokens[:seq_len]  # Truncate if longer than max_len

    # Pad to max_len
    if len(tokens) < seq_len:
        tokens += [pad_value] * (seq_len - len(tokens))

    return torch.tensor(tokens, dtype=torch.long)

def process_molecules_fixed_length(df, column, max_len, pad_value=0):
    """
    Vectorized fixed-length processing of numeric SELFIE strings.
    Returns a list of torch tensors, one per molecule.
    """
    return df[column].apply(lambda s: fast_parse_and_pad(s, max_len, pad_value)).tolist()

In [17]:
def prepare_data_for_model(tensors, batch_size):
  """the model accept as an input list of tensors"""
  #decide which part of the tensors you want to use (more = slower training but more accurate)
  tensors_for_model = tensors
  #process the tensors to be a model-ready data
  #stack the tensors to be in one tensor of shape [N, seq_len]
  x_tensor = torch.stack(tensors_for_model)
  #use pytorch approach of creating a dataset object of the big tensor we created before
  dataset = TensorDataset(x_tensor)
  #load the data using the needed batch_size, while shuffling the data every epoch
  dataloader = DataLoader(dataset, batch_size = batch_size, shuffle=True)
  return dataloader

#Model training and saving paramaters to drive

In [36]:
#the main VAE class, inherits from nn.Module
class SelfiesVAE(nn.Module):
    #init gets the base properties of the class
    #vocab_size = number of tokens in the vocabulary (of the selfies)
    #pad_idx = the token used to represent the padding
    def __init__(self, vocab_size, embedding_dim, hidden_dim, latent_dim, pad_idx,seq_len):
        #super().__init__() is needed for inheritance of nn.Module properties
        super().__init__()
        #the tensor by itself does not contribute any information - or chemical inforamtion, hence an embedding is needed for each SELFIE
        #better than one-hot due to lower dim. vector and more information can be passed through embeddings
        #padding_idx helps the embedding to know which index was used for padding the tensor - to not count them in the backprop
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        #after the embedding layer, the data is encoded into rnn (gru)
        self.encoder_rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        #then, the mu and logvar are generated - latent space
        self.to_mu = nn.Linear(hidden_dim, latent_dim)
        self.to_logvar = nn.Linear(hidden_dim, latent_dim)
        #then, the latent space is converted into the hidden linear layer
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        #after going through to the hidden layer, it is potentially possible to add act. func. -can experiment w/ that
        #finally, we decode through a gru layer
        self.decoder_rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        #then - we finally have an output through a linear layer
        self.output_layer = nn.Linear(hidden_dim, vocab_size)
        #the fixed lenght of a sequence (molecule), defined in the EDA steps
        self.seq_len = seq_len
        #padding index = 0
        self.pad_idx = pad_idx


    def encode(self, x):
        #x.long() makes the input in the correct format (int) into the embedding, embedding layer accepts integers
        x_embed = self.embedding(x.long())
        #gets the embedded and encode it through gru
        #the output of the gru is outputs, h - but we just want the h, not all the outputs. we are interested in the summery vector of all the timesteps in the gru
        _, h = self.encoder_rnn(x_embed)
        #take just the needed dimentions
        h = h.squeeze(0)
        #get the mu out of h
        mu = self.to_mu(h)
        #get the logvar out of h
        logvar = self.to_logvar(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        #reparametrization trick - using z = mu + std*eps~N(0,1), this allows to backprop through mu,std - while eps~N(0,1) stays the same
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, seq_len):
        #first, the z latent vector is decoded through linear layer that maps it into the shpe expected by the gru
        hidden = self.latent_to_hidden(z).unsqueeze(0)
        #get the batch size through z(0) - z is [batch_size, latent_vec_size]
        batch_size = z.size(0)
        #we need an initial input token to insert the gru - using the pad_idx as initial token
        #the shape of the initial input is [batch_size,1]
        #torch.long to match the expected type of embedded, then sending it to device
        #the "dummy input token" is common usage as an initializer of the gru
        input_token = torch.full((batch_size, 1), self.pad_idx, dtype=torch.long, device=z.device)
        outputs = []

        #each time the gru predicts the next token, so we iterate on each token in the seq_len
        for _ in range(seq_len):
            #get the embdedding of the input token - to get embedding vector of the token
            embed = self.embedding(input_token)
            #get the prediction (out) of the next token from the gru - based on the previous token
            out, hidden = self.decoder_rnn(embed, hidden)
            #logits are with shape [batch_size, vocab_size] meaning it maps the probability of each token from the vocabulary based on the output from the gru
            #now, instead of each token we have a vector w/ probability of each possible token. remember we have a batch each time so its [batch_size, vocab_size]
            logits = self.output_layer(out.squeeze(1))
            #save the logits of each timestep
            outputs.append(logits)
            input_token = logits.argmax(dim=1, keepdim=True)  # Greedy decoding
        #the results is stacked list of the output logits for each timestep
        return torch.stack(outputs, dim=1)  # [B, L, V]


    def forward(self, x):
      #forward is important func. - as it inherits from nn.Module to be activates once the model is called
      #the output of the forward is the logits (distr. for each token in each timestep) after the decoding, plus the mu,std of the latent space
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat_logits = self.decode(z, x.size(1))
        return x_hat_logits, mu, logvar


# === Loss Function ===

def vae_loss(x_hat_logits, x_true, mu, logvar, pad_idx):
  #we use the cross_entropy as the reconstruction loss - makes the most sense
  #here we do it token by token
  #the -1 flattens it
  #ignoring the pad index for loss calc ofc
  #
    recon_loss = F.cross_entropy(
        x_hat_logits.view(-1, x_hat_logits.size(-1)),
        x_true.view(-1),
        ignore_index=pad_idx,
        reduction='mean'
    )
    #also, we use the kl divergence as a regularization that the latent dist. behave like a gaussian dist. - keepin it as smooth as possible
    kl_div = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_div, recon_loss, kl_div


# === Training Setup ===

def train_vae(model, dataloader, optimizer, device, pad_idx, num_epochs):
    #sets the params into training setup
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        total_recon = 0
        total_kl = 0
        #loop is an approach to define dataloader just with tdqm method(for process bars)
        loop = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
        #looping through each batch
        for (x_batch,) in loop:
            x_batch = x_batch.to(device)
            #first resetting the gradients in the optimizer - independent each batch
            optimizer.zero_grad()
            #calc the x_hat, mu,var with model(batch) -> applying forward method
            x_hat_logits, mu, logvar = model(x_batch)
            #calc loss of the output
            loss, recon, kl = vae_loss(x_hat_logits, x_batch, mu, logvar, pad_idx)
            #do backprop based on the loss
            loss.backward()
            #update the model weights properly to the backprop
            optimizer.step()
            #collecting info for each loop to estimate how the model did over epochs
            #loss is a tensor, hence by .item() you get the number only
            total_loss += loss.item()
            total_recon += recon.item()
            total_kl += kl.item()
            loop.set_postfix(loss=loss.item(), recon=recon.item(), kl=kl.item())

        print(f"Epoch {epoch+1} - Loss: {total_loss:.2f}, Recon: {total_recon:.2f}, KL: {total_kl:.2f}")


In [37]:
def train_the_model(idx_to_token: dict,
                    seq_len: int, #len of generated molecule
                    embedding_dim: int, #embedding layer dimention
                    hidden_dim: int, #hidden layer dimention
                    latent_dim: int, #latent layer dim
                    num_epochs: int,
                    learning_rate: float,
                    batch_size: int,
                    tensors: list,
                    dataloader: DataLoader, #setup of data to the model
                    pad_idx = 0 #padding layer, to indicate the model to ignore it in the loss
                    ):
    #vocab size is the size of your unique tokens in your index, including padding idx
    vocab_size = len(idx_to_token)
    #set up the model with the dimenstions mentioned before
    model = SelfiesVAE(vocab_size, embedding_dim, hidden_dim, latent_dim, pad_idx,seq_len)
    #set up the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #send model to device
    model = SelfiesVAE(vocab_size, embedding_dim, hidden_dim, latent_dim, pad_idx,seq_len).to(device)
    #set up the ADAM optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # Train the model
    train_vae(model, dataloader, optimizer, device, pad_idx, num_epochs)
    return model

In [21]:
def save_model(model):
  #saving the model paramaters in drive to re-use them later on for further experiment
  # save only the model parameters (recommended)
  torch.save(model.state_dict(), 'vae_selfies_all_data_exp.pt')
  #download to the local machine (optional)
  from google.colab import files
  files.download('vae_selfies_all_data_exp.pt')
  #save to google drive
  torch.save(model.state_dict(), '/content/drive/MyDrive/vae_selfies_all_data_exp.pt')

#Loading the model and analyzing

In [22]:
def load_trained_model(vocab_size: int, embedding_dim: int, hidden_dim: int,
                       latent_dim: int, pad_idx: int, seq_len: int, model_name: str):
  """Once finished training, we can load the model annd analyze it"""
  #set up the model archetecture
  model = SelfiesVAE(vocab_size, embedding_dim, hidden_dim, latent_dim, pad_idx,seq_len)
  #set up the device
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  #send model to device
  model = SelfiesVAE(vocab_size, embedding_dim, hidden_dim, latent_dim, pad_idx,seq_len).to(device)
  #load the model paramaters
  model.load_state_dict(torch.load(f'/content/drive/MyDrive/{model_name}'))
  return model

In [24]:
def generate_molecule_from_trained_model(model, seed_idx, seq_len, idx_to_token, device, tensor_idx):
  """once model is already trained, we can do analysis / research of its operations / latent space"""
  #choose the index of the tensor of the seed molecule
  mol = tensors[tensor_idx]
  print(mol)
  #send to device
  x = mol.unsqueeze(0).to(device)
  #use eval mode - to not accidently activate all the behaviors of .train() mode
  # make sure model is in eval mode - no_grad saves memory and time, ensure inference mode
  model.eval()
  with torch.no_grad():
      #encode the mu, logvar that represents your molecule
      mu, logvar = model.encode(x)
      z = model.reparameterize(mu, logvar)  # shape: [1, latent_dim]
      #decode your sampled molecule from the latent space
      logits = model.decode(z, seq_len)
      token_ids = logits.argmax(dim=2)[0]  # [seq_len]
      #get the original molecule's representation to compare to the generated
      generated_tokens = [idx_to_token[str(i.item())] for i in token_ids]
      #get the chemical represnetation of the generated molecule
      original_mol = [idx_to_token[str(i.item())] for i in x[0]]
      return generated_tokens, original_mol


#Main running function

In [35]:
if __name__== "__main__":
  #load the data
  main_df, idx_to_tokens, tokens_to_idx = load_data_and_selfies_indexing_jsons(file_name = 'chembl_relevant_info_with_numeric_selfies.csv',
                                                                               idx_to_token_file = 'idx_to_token',
                                                                               token_to_idx_file = 'token_to_indx')
  #setting up the model's paramaters
  vocab_size = len(idx_to_tokens)
  #seq_len is the maximum len of a tensor for training and generation of the vae
  seq_len = int(analyze_data_distr_for_optimal_paramaters(main_df = main_df, percentile = 0.95))
  embedding_dim = 256
  hidden_dim = 256
  latent_dim = 64
  #padding value for shorter molecules (then the seq_len)
  pad_idx = 0
  num_epochs = 20
  batch_size = 64
  learning_rate = 1e-3
  #get the tensors ready for a model
  tensors = process_molecules_fixed_length(df = main_df, column ='numeric_selfies', max_len = seq_len, pad_value = pad_idx)
  #prepare the data to be model - ready
  dataloader = prepare_data_for_model(tensors = tensors, batch_size = batch_size)
  #train the model
  model = train_the_model(idx_to_token = idx_to_tokens,
                          seq_len = seq_len,
                          embedding_dim = embedding_dim,
                          hidden_dim = hidden_dim,
                          latent_dim = latent_dim,
                          num_epochs = num_epochs,
                          learning_rate = learning_rate,
                          batch_size = batch_size,
                          tensors = tensors,
                          dataloader = dataloader,
                          pad_idx = pad_idx)


TypeError: train_vae() missing 4 required positional arguments: 'learning_rate', 'batch_size', 'tensors', and 'dataloader'

drive  sample_data  thesis
